In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# %%bash
# MINICONDA_INSTALLER_SCRIPT=Miniconda3-4.5.4-Linux-x86_64.sh
# MINICONDA_PREFIX=/usr/local
# wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
# chmod +x $MINICONDA_INSTALLER_SCRIPT
# ./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

In [ ]:
import gzip
import time
from multiprocessing import Pool
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import utils.gcs_utils as gcs
import utils.spark_utils as sprk
from utils.proteins import *

In [ ]:
import numpy as np
from bio_embeddings.embed import SeqVecEmbedder, ProtTransBertBFDEmbedder  

In [ ]:
home_path = "/home/jupyter/pss"

In [ ]:
files = gcs.list_keys("UP000005640_9606_HUMAN/cif")

## Parse sequences and atom sites
*reduce files that are multipart*

In [ ]:
def parse_cif_file_in_cloud(key):
    content = gcs.download_gzip_to_string(key)
    parsed_cif = parse_cif(content)
    file_name = key.strip(".cif.gz").split("/")[-1]

    sequence = get_protein_sequence_from_cif(parsed_cif)
    sequence['protein_filename'] = file_name
    sequence['protein_id'] = get_protein_id_from_filename(file_name)
    sequence['confidence_pLDDT'] = get_global_confidence_from_cif(parsed_cif)

    atoms = get_atom_sites_from_cif(parsed_cif)
    atoms['protein_filename'] = file_name
    atoms['protein_id'] = get_protein_id_from_filename(file_name)
    local_confidence = get_local_confidence_from_cif(parsed_cif)
    atoms = sort_by_file_number_and_index(join_atoms_with_confidence(atoms, local_confidence))
    return sequence, atoms

In [ ]:
sequences = []

# could potentially use spark here with wholeTextFile reader but that is not necessary in our usecase
for i in tqdm(range(len(files) // 1000 + 1)):
    with Pool() as p:
        structures = p.map(parse_cif_file_in_cloud, files[i*1000:(i+1)*1000])
    sequences_part, atom_sites_part = zip(*structures)
    sequences.append(pd.concat(sequences_part))

    # this grows quickly, so we will write these in parts
    pd.concat(atom_sites_part).to_parquet(path=f"{home_path}/structure_files/atom_sites/atom_sites_part_{str(i).zfill(2)}.parquet")

In [ ]:
sequences_df = reduce_sequence_df(pd.concat(sequences))
sequences_df.to_parquet(path=f"{home_path}/structure_files/sequences/sequences.parquet")

In [50]:
sequences_df = pd.read_parquet(path=f"{home_path}/structure_files/sequences/sequences.parquet")
sequences_df.head()

,pdbx_db_accession,db_code,db_name,protein_id,pdbx_seq_one_letter_code,protein_filename
0,A0A024R1R8,A0A024R1R8_HUMAN,UNP,A0A024R1R8,MSSHEGGKKKALKQPKKQAKEMDEEEKAFKQKQKEEQKKLEVLKAK...,AF-A0A024R1R8-F1-model_v1
1,A0A024RBG1,NUD4B_HUMAN,UNP,A0A024RBG1,MMKFKPNQTRTYDREGFKKRAACLCFRSEQEDEVLLVSSSRYPDQW...,AF-A0A024RBG1-F1-model_v1
2,A0A024RCN7,A0A024RCN7_HUMAN,UNP,A0A024RCN7,MERSFVWLSCLDSDSCNLTFRLGEVESHACSPSLLWNLLTQYLPPG...,AF-A0A024RCN7-F1-model_v1
3,A0A075B6H5,A0A075B6H5_HUMAN,UNP,A0A075B6H5,METVVTTLPREGGVGPSRKMLLLLLLLGPGSGLSAVVSQHPSRVIC...,AF-A0A075B6H5-F1-model_v1
4,A0A075B6H7,KV37_HUMAN,UNP,A0A075B6H7,MEAPAQLLFLLLLWLPDTTREIVMTQSPPTLSLSPGERVTLSCRAS...,AF-A0A075B6H7-F1-model_v1


In [51]:
atom_sites_df = pd.read_parquet(path=f"{home_path}/structure_files/atom_sites/atom_sites_part_00.parquet")
atom_sites_df.head()

,group_PDB,id,type_symbol,label_atom_id,label_alt_id,label_comp_id,label_asym_id,label_entity_id,label_seq_id,pdbx_PDB_ins_code,...,auth_asym_id,auth_atom_id,pdbx_PDB_model_num,pdbx_sifts_xref_db_acc,pdbx_sifts_xref_db_name,pdbx_sifts_xref_db_num,pdbx_sifts_xref_db_res,protein_filename,protein_id,confidence_pLDDT
0,ATOM,1,N,N,,MET,A,1,1,None,...,A,N,1,A0A024R1R8,UNP,1,M,AF-A0A024R1R8-F1-model_v1,A0A024R1R8,59.87
1,ATOM,2,C,CA,,MET,A,1,1,None,...,A,CA,1,A0A024R1R8,UNP,1,M,AF-A0A024R1R8-F1-model_v1,A0A024R1R8,59.87
2,ATOM,3,C,C,,MET,A,1,1,None,...,A,C,1,A0A024R1R8,UNP,1,M,AF-A0A024R1R8-F1-model_v1,A0A024R1R8,59.87
3,ATOM,4,C,CB,,MET,A,1,1,None,...,A,CB,1,A0A024R1R8,UNP,1,M,AF-A0A024R1R8-F1-model_v1,A0A024R1R8,59.87
4,ATOM,5,O,O,,MET,A,1,1,None,...,A,O,1,A0A024R1R8,UNP,1,M,AF-A0A024R1R8-F1-model_v1,A0A024R1R8,59.87
